In [3]:
import pickle
import os

from glob import glob
from gensim import models
from nltk.corpus import stopwords
from FlaskApp.transform import *
from googletrans import Translator

import pandas as pd
import numpy as np

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [7]:
filepaths = glob('data/scraped/job_*')
filenames = [filepath.split('\\')[-1].split('.')[0] for filepath in filepaths]

In [21]:
jobs = {}
for filepath in filepaths:
    filename = filepath.split('\\')[-1].split('.')[0]
    with open(filepath, 'rb') as file:
        loaded = pickle.load(file)
        for job in loaded:
            if 'Type' not in job.keys():
                job['Type'] = None
            if type(job['Description']) != str:
                print(type(job['Description']))
        jobs[filename] = loaded

In [67]:
import re

In [101]:
df_job = pd.DataFrame({
    'Title': [job['Title'] for key in jobs for job in jobs[key]],
    'Company': [job['Company'] for key in jobs for job in jobs[key]],
    'Area': [job['Area'] for key in jobs for job in jobs[key]],
    'Type': [job['Type'] for key in jobs for job in jobs[key]],
    'Skills': [job['Skills'] for key in jobs for job in jobs[key]],
    'Description': [job['Description'] for key in jobs for job in jobs[key]],
})

df_job.Description = df_job.Description.apply(lambda x: x.lower().split('tentang pekerjaan ini')[-1].split('pronunciation')[0])
df_job.Description = df_job.Description.apply(lambda x: re.sub('[^a-zA-Z0-9&#.+-]', ' ', x))

df_job.Skills = df_job.Skills.apply(lambda x: ' '.join(x) if x != None else '').map(str.lower).apply(lambda x: re.sub('[^a-zA-Z0-9&#.+-]', ' ', x))

In [103]:
df_job.tail()

,Title,Company,Area,Type,Skills,Description
6414,Senior Full-Stack Engineer,Turing,Indonesia,Jarak Jauh,asp.net asp.net mvc back-end web development ef mern stack model-view-controller mvc oauth sql stored procedures web applications,tentang kami lakeba group adalah salah satu perusahaan teknologi baru yang paling inovatif dan menarik di australia diakui secara internasional oleh the financial times sebagai perusahaan pertumbuhan tinggi asia pasifik selama dua tahun berturut-turut. tim teknik global kami mengembangkan solusi al yang inovatif dan inovatif untuk beberapa layanan keuangan paling menarik dan bergengsi di dunia serta klien perusahaan terkemuka. membangun pertumbuhan dua digit selama bertahun-tahun kami bersemangat untuk mengembangkan tim penjualan kami untuk mempercepat pertumbuhan baik di dalam negeri maupun di as asia tenggara india dan eropa. dalam beberapa tahun terakhir lakeba telah memperluas operasinya secara global dan telah mendirikan kantor di beberapa negara termasuk australia amerika serikat dan indonesia. keputusan untuk membuka kantor di indonesia didorong oleh beberapa faktor termasuk kumpulan besar pekerja berbakat dan terampil lingkungan bisnis yang menguntungkan dan lokasi yang strategis di asia tenggara. kami percaya pada kekuatan pengetahuan kolektif yang terdiri dari tempat kerja inklusif kami investasi kami inovasi kami dan filosofi pemikiran desain kami. memimpin dengan tujuan lakeba memberikan hasil transformatif yang merekayasa teknologi masa depan mempercepat bisnis untuk hari esok yang lebih baik. tentang peran kami mencari pengembang full stack mern yang terampil yang akan bertanggung jawab untuk mengembangkan dan atau merancang produk berbasis saas untuk usaha kami. anda akan bekerja sama dengan pengembang lain dalam membuat memelihara dan memperbarui produk kami. kandidat ideal kami adalah seseorang yang bersedia tumbuh bersama kami. agar anda berhasil dalam peran ini anda harus mahir dalam pengembangan back-end dengan inti .net .net pengetahuan dan pengalaman yang kuat dalam pemrograman aplikasi web menggunakan sql. deskripsi pekerjaan ini adalah kesempatan yang menarik bagi pengembang backend untuk bergabung dengan perusahaan global yang menyenangkan dan berkembang pesat. posisi tersebut akan melapor kepada pemimpin tim pengembangan dan akan bekerja sama erat dengan arsitek solusi. tanggung jawab utama mengembangkan atau memelihara layanan web atau aplikasi web menggunakan tumpukan teknis microsoft termasuk c# asp.net mvc prinsip rest sql server berkolaborasi dengan tim devops dalam mengidentifikasi memperbaiki masalah produksi merancang dan mengimplementasikan integrasi dengan platform pihak ketiga berikut praktik terbaik. melakukan tinjauan kode berkolaborasi dengan arsitek solusi dalam membuat poc proposal solusi dan membantu dengan arsitektur aplikasi berhubungan dengan tim pengembangan front end atau seluler pelaporan berkelanjutan dan pembaruan proyek ke semua pemangku kepentingan yang diperlukan keterampilan yang dibutuhkan mengembangkan dan hosting layanan web kesombongan dan integrasi dengan aplikasi web azure. pengalaman yang kuat dalam berbagai jenis otentikasi otentikasi oauth otentikasi berbasis token openid otentikasi berbasis klaim dan azure active directory c# asp.net layanan web server sql kueri tingkat lanjut penyesuaian kinerja prosedur tersimpan arsitektur mvc kerangka entitas. pengetahuan suara dalam push notifications signalr pengalaman kerja dalam ci cd dan build automation. harus memiliki pengetahuan ef baik untuk memahami teknologi web seperti ajax angular js jquery wcf layanan lainnya dll. pengetahuan inti .net menguntungkan keistimewaan di lakeba bekerja dengan tim berorientasi kinerja yang didorong oleh kepemilikan dan terbuka untuk eksperimen dan teknologi baru produk yang akan anda kerjakan digunakan oleh bank besar di seluruh dunia belajar merancang sistem untuk akurasi efisiensi dan skalabilitas tinggi fokus pada hasil kerja yang berkualitas. budaya ju

In [106]:
df_job.isna().sum()

Title          0
Company        0
Area           0
Type           0
Skills         0
Description    0
dtype: int64

In [105]:
df_job.fillna('', inplace=True)

In [107]:
df_job['Text'] = df_job.Skills + ' ' + df_job.Description

In [108]:
id_stopwords = stopwords.words('indonesian')
en_stopwords = stopwords.words('english')

In [109]:
nltk_stopwords = id_stopwords + en_stopwords

In [110]:
df_job.Text = df_job.Text.apply(lambda x: ' '.join([i for i in x.split() if i not in nltk_stopwords]))

In [111]:
df_job.head(1)

,Title,Company,Area,Type,Skills,Description,Text
0,Senior 3D Civil Designer,Worley,"Sulawesi Barat, Indonesia",Di Kantor,3d computer aided design 3d cad autodesk civil 3d civil engineering design civil engineering drafting computer-aided design cad constructability earthworks engineering drawings health and safety executive hse representational state transfer rest,lebih dari 50.000 orang. 49 negara. lebih dari 120 lokasi kantor. kami bukan hanya insinyur. kami adalah tim global yang terdiri dari ilmuwan data konsultan pekerja konstruksi dan inovator semuanya bekerja untuk menciptakan hari esok yang lebih baik. setiap hari kami membantu pelanggan di bidang energi bahan kimia dan sumber daya selangkah lebih dekat untuk memecahkan masalah terberat di planet kita. perubahan iklim. transisi energi. transformasi digital. dan bagaimana kita dapat menghadirkan dunia yang lebih berkelanjutan. apa pun ambisi anda ada jalan untuk anda di sini. dan tidak ada penghalang untuk kesuksesan karier potensial anda. untuk informasi lebih lanjut kunjungi worley.com akuntabilitas utama bertanggung jawab atas praktik kerja yang aman baik secara pribadi maupun dalam desain mereka menggunakan kerangka kerja worley pastikan mereka memahami dan menerapkan sistem manajemen mutu worley pastikan pekerjaan desain sesuai dengan spesifikasi kode dan standar yang berlaku pastikan prosedur pemeriksaan dipatuhi sebelum penerbitan gambar dan dokumen untuk memastikan tidak ada pengerjaan ulang karena kesalahan dalam desain menyiapkan gambar teknik dan dokumentasi mempromosikan komunikasi interdisipliner pada proyek memastikan ruang lingkup proyek anggaran dan jadwal untuk pekerjaan desain dan penyusunan dikomunikasikan secara efektif dan dipenuhi pada proyek. alokasi pekerjaan penyusunan untuk desainer junior dalam proyek pendampingan dan pengawasan desainer junior persyaratan gelar diploma teknik di bidang yang relevan atau setara pengalaman minimal 5 tahun dalam desain sipil 3d untuk pekerjaan tanah pengetahuan dalam desain untuk konstruksi komisioning operasi dan pemeliharaan memiliki pengalaman dan keterampilan untuk menetapkan standar dan mendorong anggota tim lainnya untuk unggul dalam hse kompeten dalam perangkat lunak cad 3d sipil keterampilan komunikasi yang sangat baik bekerja sebagai bagian dari tim multidisiplin. keterampilan manajemen waktu yang sangat baik dan kemampuan untuk bekerja secara mandiri harus dapat pindah ke lokasi proyek untuk bekerja yang kami tawarkan kepada anda memberikan budaya yang gesit dan peluang kerja yang menantang menghasilkan lingkungan kerja yang positif. orang-orang kami menikmati paket kompensasi yang kompetitif dan lingkungan kerja yang menyenangkan personal kolaboratif dan aman. kami menghargai kesempatan kerja yang setara dan berkomitmen untuk mempromosikan keadilan kesetaraan dan keragaman. kebijakan kami adalah melakukan pemeriksaan latar belakang untuk semua kandidat yang menerima tawaran pekerjaan dengan kami. jika anda memiliki hasrat dan bakat untuk mengikutinya inilah saatnya untuk menguji batas kemampuan anda. temukan masa depan anda dengan worley. company worley lokasi utama pekerjaan idn-sulawesi jadwal sipil purna waktu jenis pekerjaan tingkat pekerjaan karyawan executive job posting apr 17 2023 tanggal unposting 17 mei 2023 reporting manager title study manager,3d computer aided design 3d cad autodesk civil 3d civil engineering design civil engineering drafting computer-aided design cad constructability earthworks engineering drawings health safety executive hse representational state transfer rest 50.000 orang. 49 negara. 120 lokasi kantor. insinyur. tim global ilmuwan data konsultan pekerja konstruksi inovator menciptakan esok baik. membantu pelanggan bidang energi bahan kimia sumber daya selangkah memecahkan terberat planet kita. perubahan iklim. transisi energi. transformasi digital. menghadirkan dunia berkelanjutan. ambisi jalan sini. penghalang kesuksesan karier potensial anda. informasi kunjungi worl

In [112]:
dataset = df_job.Text.map(str.split).values

data = []
for word in dataset:
    data.append(word)

def tagged_document(list_of_listwords, tags=None):
    if tags is not None:
        for x, listwords in zip(tags, list_of_listwords):
            yield models.doc2vec.TaggedDocument(listwords, [x])
    else:
        for x, listwords in enumerate(list_of_listwords):
            yield models.doc2vec.TaggedDocument(listwords, [x])

data_train = list(tagged_document(data, df_job.Title.values))


In [121]:
d2v_model = models.doc2vec.Doc2Vec(vector_size=120, min_count=5, epochs=60, window=2)
d2v_model.build_vocab(data_train)

2023-05-04 15:19:58,485 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d120,n5,w2,mc5,s0.001,t3>', 'datetime': '2023-05-04T15:19:58.485675', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-05-04 15:19:58,488 : INFO : collecting all words and their counts
2023-05-04 15:19:58,489 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-05-04 15:19:58,648 : INFO : collected 22095 word types and 3667 unique tags from a corpus of 6419 examples and 1153494 words
2023-05-04 15:19:58,649 : INFO : Creating a fresh vocabulary
2023-05-04 15:19:58,684 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 11330 unique words (51.28% of original 22095, drops 10765)', 'datetime': '2023-05-04T15:19:58.684428', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 

In [122]:
list(d2v_model.wv.index_to_key)

['pengalaman',
 'tim',
 'bisnis',
 'kerja',
 'perusahaan',
 'memiliki',
 'pekerjaan',
 'penjualan',
 'pelanggan',
 'produk',
 '&',
 'keterampilan',
 'manajemen',
 'layanan',
 'komunikasi',
 'data',
 'pengembangan',
 'kemampuan',
 'proyek',
 'teknologi',
 'mengembangkan',
 'pemasaran',
 'klien',
 'pengetahuan',
 'proses',
 'kuat',
 'membangun',
 'desain',
 'solusi',
 'keuangan',
 'persyaratan',
 'indonesia',
 'terkait',
 'membantu',
 'mengelola',
 'sistem',
 'bahasa',
 'sesuai',
 'tanggung',
 'tujuan',
 'aplikasi',
 'mendukung',
 'industri',
 'peluang',
 'pasar',
 'bidang',
 'bertanggung',
 'hubungan',
 'meningkatkan',
 'peran',
 'informasi',
 'strategi',
 'utama',
 'lingkungan',
 '-',
 'perangkat',
 'karyawan',
 'platform',
 'teknis',
 'kebutuhan',
 'orang',
 'mendorong',
 'kami.',
 'gelar',
 'web',
 'operasi',
 'internal',
 'mencari',
 'nilai',
 'lunak',
 'digital',
 'description',
 'sarjana',
 'inggris',
 'laporan',
 'status',
 'global',
 'tamu',
 'budaya',
 'kesempatan',
 'mitra',
 

In [123]:
d2v_model.train(data_train, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

2023-05-04 15:19:59,765 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 11330 vocabulary and 120 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2023-05-04T15:19:59.765686', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2023-05-04 15:20:00,404 : INFO : EPOCH 0: training on 1153494 raw words (1079109 effective words) took 0.6s, 1694473 effective words/s
2023-05-04 15:20:01,010 : INFO : EPOCH 1: training on 1153494 raw words (1079064 effective words) took 0.6s, 1789259 effective words/s
2023-05-04 15:20:01,608 : INFO : EPOCH 2: training on 1153494 raw words (1079223 effective words) took 0.6s, 1808929 effective words/s
2023-05-04 15:20:02,194 : INFO : EPOCH 3: training on 1153494 raw words (1079407 effective words) took 0.6s, 1849799 effective words/s
2023-05-04 15:20:02,785 : 

In [124]:
linkedin_words = d2v_model.wv.index_to_key
linkedin_vectors = d2v_model.wv.vectors

with open('data/linkedin_vectors.tsv', 'w', encoding='utf-8') as f:
    for i, word in enumerate(linkedin_words):
        vector_str = '\t'.join([str(num) for num in linkedin_vectors[i]])
        f.write(f'{vector_str}\n')

with open('data/linkedin_metadata.tsv', 'w', encoding='utf-8') as f:
    for word in linkedin_words:
        f.write(f'{word}\n')